# Data Creation/Training Notebook 

In [11]:
import os
import pandas as pd
import numpy as np
import datetime as dt

# custom
from utilities import extract_position_time
from image_generator import plot_sky, create_times_array, get_waypoints, create_times_array, DRCalc

# skyfield
from skyfield.api import load, utc, Topos

# I. Creating Training Images

Create Waypoints 

In [16]:
# starting and ending latitude and longitude
start = (22.0250, 88.0583)
end = (11.6833, 92.7167)

In [17]:
# create waypoints
waypoints = get_waypoints(start, end, n=100, list_of_mile_displacements=[x for x in range(0, 10, 1)])

Number of unique waypoints:  1919


Create Times

In [18]:
# start and end time
start_time = dt.datetime(2023, 11, 22, 20, 0, 0)
end_time = dt.datetime(2023, 11, 23, 0, 0, 0)

# create times array evenly spaced by specified time interval n
times = create_times_array(start_time, end_time, n=1/60)
# print(times)

In [30]:
# print(times[0])

<Time tt=[2460271.334134074 ... 2460271.500800741] len=14401>


In [45]:
# for p in waypoints:
#     for t in times[0]:
#         print(t)

### Iterate Through Times for Each Position

Due to the large number of plots it is necessary to create the the images in batches to avoid memory issues.

In [46]:
# p = waypoints[0]
# p

In [47]:
# observer = Topos(p[0], p[1])
# observer

In [48]:
# t = times[0][0]
# p = observer.at(t)
# p

In [49]:
# for p in waypoints:
#     for t in times[0][5:8]:
#         print(t)
#         plot_sky(t, observer, cloud_cover = 0, img_directory = "data/kol_andaman/")

In [19]:
batch = range(0,8)
print(f'Creating {len(waypoints)*(len(times[0][batch]))} sky images, for grid {start} to {end}')

def batch_image_generator(batch, waypoints, times, base_directory, cloud_cover = 0):
    print("function called")
    """
    Generates sky images for a given batch of waypoints and times

    Args:
        batch (range): time range to generate images for
        waypoints (np.array): list of waypoints
        times (skyfield timelib.Time): array of times
        base_directory (directory): directory to hold cloud cover image directories
        cloud_cover (int, optional): cloud cover to generate images for. Defaults to 0.
    """
    for p in waypoints:
        for t in times[0]:
            # print(t)
            observer = Topos(p[0], p[1])
            # create a directory for each cloud cover
            directory = base_directory + str(0) + '/'
            os.makedirs(directory, exist_ok=True)
            plot_sky(t, observer, cloud_cover = cloud_cover, img_directory = directory)

# batch_image_generator(batch, waypoints, times, base_directory = '/Users/DELL/PycharmProjects/Deep-Learning-Celestial-Navigation/data/kol_andaman/', cloud_cover = 0)

Creating 15352 sky images, for grid (22.025, 88.0583) to (11.6833, 92.7167)


# II. Missing Images and Out of Order Image Sequences

In [20]:
# training
import os
files_train = os.listdir('/Users/DELL/PycharmProjects/Deep-Learning-Celestial-Navigation/data/kol_andaman/0')
files_train = [f for f in files_train if f.endswith('.png')]

# files is a list of all the files in train and valid directories
files = files_train 

# print number of files
print(f'Number of files: {len(files)}')

Number of files: 15506


## Create Dataframe of Times and Positions

In [21]:
data_df = pd.DataFrame({'times':[extract_position_time(f)[1] for f in files]})
data_df['positions'] = [extract_position_time(f)[0] for f in files]
data_df.sort_values(by = 'times', inplace = True)
complete_times = data_df['times'].unique()

In [27]:
# for each position, find which times are missing
def fill_missing():
    # iterate for each position and find which times are missing
    missing_times = []
    for position in data_df['positions'].unique():
        # get all times for this position
        times = data_df[data_df['positions'] == position]['times'].unique()
        # find which times are missing
        missing = np.setdiff1d(complete_times, times)
        if len(missing) > 0:
            missing = [pd.to_datetime(m) for m in missing]
            # get times array
            times_array = create_times_array(missing[0], missing[-1], 6)
            for time in times_array:
                observer = Topos(position[0], position[1])
                plot_sky(t = time, observer = observer, cloud_cover= 0/8, img_directory='/Users/DELL/PycharmProjects/Deep-Learning-Celestial-Navigation/data/kol_andaman/train/')

Every position now has an image for every time, in the correct sequence!

In [23]:
data_df.groupby('times').count()

,positions
times,
2023-11-22 20:00:00,2
2023-11-22 20:00:01,2
2023-11-22 20:00:02,2
2023-11-22 20:00:03,2
2023-11-22 20:00:04,2
...,...
2023-11-22 23:59:56,1
2023-11-22 23:59:57,1
2023-11-22 23:59:58,1


# III. Create Validation Images

In [28]:
# Use DRcalc to calculate the position of the ship every 15 minutes for 4 hours
positions = []
lat = 22.0250
long = 88.0583
positions.append(np.array([lat, long]))
for i in range(len(times[1])-1):
    ship = DRCalc(lat, long, dt.timedelta(minutes=1/60).total_seconds(), 110, 20)
    lat = ship.drlatfwds
    long = ship.drlongfwds

    positions.append(np.array([lat, long]))

print(len(positions))
# show the first 10 positions
positions[:10]

14401


[array([22.025 , 88.0583]),
 array([22.02496833, 88.05839331]),
 array([22.02493666, 88.05848662]),
 array([22.02490499, 88.05857994]),
 array([22.02487333, 88.05867325]),
 array([22.02484166, 88.05876656]),
 array([22.02480999, 88.05885987]),
 array([22.02477832, 88.05895319]),
 array([22.02474665, 88.0590465 ]),
 array([22.02471498, 88.05913981])]

In [ ]:
def create_voyage_images(times, positions, directory, cloud_cover=0/8):
    """Creates validation images for a given set of times and positions

    Args:
        times (Skyfield.timelib.Time): Skyfield time array.
        positions (np.array): Array of positions.
        directory (directrory): Directory to hold validation images. 
        cloud_cover (int, optional): Cloud cover of sky images. Defaults to 0/8.
    """
    for i, position in enumerate(positions):
        observer = Topos(latitude_degrees = position[0], longitude_degrees=position[1])
        print('Creating sky For :',times[1][i], position)
        plot_sky(times[0][i], observer, cloud_cover=0/8, img_directory=directory)

create_voyage_images(times, positions, directory='/Users/DELL/PycharmProjects/Deep-Learning-Celestial-Navigation/data/kol_andaman/0_val/')

Creating sky For : 2023-11-22 20:00:00+00:00 [22.025  88.0583]
Creating sky For : 2023-11-22 20:00:01+00:00 [22.02496833 88.05839331]
Creating sky For : 2023-11-22 20:00:02+00:00 [22.02493666 88.05848662]
Creating sky For : 2023-11-22 20:00:03+00:00 [22.02490499 88.05857994]
Creating sky For : 2023-11-22 20:00:04+00:00 [22.02487333 88.05867325]
Creating sky For : 2023-11-22 20:00:05+00:00 [22.02484166 88.05876656]
Creating sky For : 2023-11-22 20:00:06+00:00 [22.02480999 88.05885987]
Creating sky For : 2023-11-22 20:00:07+00:00 [22.02477832 88.05895319]
Creating sky For : 2023-11-22 20:00:08+00:00 [22.02474665 88.0590465 ]
Creating sky For : 2023-11-22 20:00:09+00:00 [22.02471498 88.05913981]
Creating sky For : 2023-11-22 20:00:10+00:00 [22.02468331 88.05923312]
Creating sky For : 2023-11-22 20:00:11+00:00 [22.02465165 88.05932643]
Creating sky For : 2023-11-22 20:00:12+00:00 [22.02461998 88.05941975]
Creating sky For : 2023-11-22 20:00:13+00:00 [22.02458831 88.05951306]
Creating sky F